In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nbimporter
from utils import *

In [2]:
def load_and_process_data(data1, data2, data3, data4, common_key, target_column):
    """Charge, fusionne et nettoie les fichiers CSV."""
    caract = pd.read_csv(data1, sep=";", low_memory=False)
    lieux = pd.read_csv(data2, sep=";", low_memory=False)
    usagers = pd.read_csv(data3, sep=";", low_memory=False)
    vehicules = pd.read_csv(data4, sep=";", low_memory=False)
    
    merged_df = caract.merge(lieux, on=common_key, how="inner") \
                      .merge(usagers, on=common_key, how="inner") \
                      .merge(vehicules, on=common_key, how="inner")
    
    print(f"Colonnes disponibles après fusion : {merged_df.columns.tolist()}")

    merged_df = merged_df.dropna(axis=1, how='all')
    
    y = merged_df[target_column]
    X = merged_df.drop(columns=[target_column, common_key])
    return X, y


In [3]:
def preprocess_data(data1, data2, data3, data4, common_key, target_column):
    print("Chargement et fusion des données...")
    X, y = load_and_process_data(data1, data2, data3, data4, common_key, target_column)

    print("\nSuppression des doublons...")
    print("Nombre de doublons avant suppression :", X.duplicated().sum())
    X = X.drop_duplicates()
    print("Nombre de doublons après suppression :", X.duplicated().sum())

    print("\nTraitement des valeurs NaN...")
    X = auto_handle_nan(X)

    print("\nSélection des colonnes numériques importantes...")
    numeric_cols_corr = correlation_matrix(X, y, threshold=0.1)
    numeric_cols_var = low_variance_filter(X, threshold=0.1)

    print("\nSélection des colonnes catégorielles importantes...")
    categorical_cols = categorical_analysis(X, y, threshold=2)

    selected_columns = list(set(numeric_cols_var + categorical_cols))
    print(f"\nColonnes finales sélectionnées : {selected_columns}")

    X_filtered = X[selected_columns]
    # final_data = pd.concat([X_filtered, y], axis=1)

    print("\nRésumé des colonnes importantes pour la prédiction :")
    print(f"Nombre de colonnes finales : {len(X_filtered.columns)}")
    print(X_filtered.head())
    return X_filtered, y